In [119]:
import pandas as pd
import numpy as np
import toml
from pathlib import Path
from sqlalchemy import create_engine

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.1%}'.format)

In [120]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]

In [121]:
# add trip type to trip tables
def prep_trip_df(trip):
    df_trip = trip.copy()
    df_trip.rename(columns={'mode':'Mode'}, inplace=True)
    df_trip['Trip Type'] = 'Non-Work'
    df_trip.loc[df_trip['dpurp'] == 'Work', 'Trip Type'] = 'Work'

    # add all tripa
    df = df_trip.copy()
    df['Trip Type'] = 'All Trips'
    df_trip = pd.concat([df_trip.sort_values('Trip Type'), df], ignore_index=True)

    return df_trip

In [122]:
person = pd.read_csv(output_path / 'agg/dash/person_geog.csv')

# counties
trip_county = prep_trip_df(pd.read_csv(output_path / 'agg/dash/mode_share_county.csv'))
# remove "Outside Region"
trip_county = trip_county[trip_county['hh_county'] != 'Outside Region']
# regional geographies
trip_rg = prep_trip_df(pd.read_csv(output_path / 'agg/dash/mode_share_rg.csv'))
# centers
trip_rgc = pd.read_csv(output_path / 'agg/dash/mode_share_rgc.csv')
# add all RGCs
df = trip_rgc[trip_rgc['hh_rgc'] != 'Not in RGC'].groupby(['dpurp','mode']).sum()[['trexpfac']].reset_index()
df['hh_rgc'] = 'All RGCs'
trip_rgc = pd.concat([df, trip_rgc])
trip_rgc = prep_trip_df(trip_rgc)

In [123]:
# equity geographies
equity_geogs = summary_config['equity_geogs']
trip_equity_geog = pd.DataFrame()
for geog in equity_geogs:
    df = prep_trip_df(pd.read_csv(output_path/ ('agg/dash/mode_share_'+geog+'.csv')))
    df['equity_geog'] = df[geog].map({1:geog,
                                      0:"NOT in "+ geog})
    
    trip_equity_geog = pd.concat([trip_equity_geog, df[['dpurp', 'Mode', 'equity_geog', 'trexpfac', 'Trip Type']]])


## regional mode share

In [124]:
df1 = trip_county.groupby(['Trip Type','Mode'],as_index=False)['trexpfac'].sum()
df2 = trip_county.groupby(['Trip Type'],as_index=False)['trexpfac'].sum()
df2.rename(columns={'trexpfac':'total_trips'}, inplace=True)

df = df1.merge(df2, on='Trip Type')
df['mode_share'] = df['trexpfac'] / df['total_trips']

df.pivot_table(columns='Mode', index='Trip Type', values='mode_share')

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
Trip Type,,,,,,,,
All Trips,1.5%,21.3%,14.1%,44.5%,1.6%,0.3%,2.2%,14.4%
Non-Work,1.5%,23.1%,15.4%,40.4%,1.8%,0.3%,2.2%,15.3%
Work,1.4%,9.4%,5.8%,71.6%,0.0%,0.6%,2.3%,8.9%


## trip mode share by home location

In [125]:
def calc_mode_share(df_trip, geog):
    
    # num trips by mode
    df1 = df_trip.groupby([geog, 'Trip Type', 'Mode'],as_index=False)['trexpfac'].sum()
    # num trips
    df2 = df_trip.groupby([geog, 'Trip Type'],as_index=False)['trexpfac'].sum()
    df2.rename(columns={'trexpfac':'total_trips'}, inplace=True)

    df = df1.merge(df2, on=[geog, 'Trip Type'])
    # mode share
    df['mode_share'] = df['trexpfac'] / df['total_trips']

    # mode share by trip type
    df_all = df.loc[df['Trip Type']=="All Trips"].pivot_table(columns='Mode', index=geog, values='mode_share')
    df_work = df.loc[df['Trip Type']=="Work"].pivot_table(columns='Mode', index=geog, values='mode_share')
    df_non_work = df.loc[df['Trip Type']=="Non-Work"].pivot_table(columns='Mode', index=geog, values='mode_share')
    # num trips by trip type
    trip_all = df.loc[df['Trip Type']=="All Trips"].pivot_table(columns='Mode', index=geog, values='trexpfac')
    trip_work = df.loc[df['Trip Type']=="Work"].pivot_table(columns='Mode', index=geog, values='trexpfac')
    trip_non_work = df.loc[df['Trip Type']=="Non-Work"].pivot_table(columns='Mode', index=geog, values='trexpfac')

    return df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work


df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work = calc_mode_share(trip_county,'hh_county')

In [126]:
df_all

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_county,,,,,,,,
King,1.7%,20.2%,13.2%,43.4%,1.5%,0.4%,3.3%,16.4%
Kitsap,1.2%,22.2%,14.3%,45.3%,1.5%,0.2%,1.3%,14.1%
Pierce,1.4%,23.1%,15.5%,45.1%,1.9%,0.2%,0.8%,12.1%
Snohomish,1.3%,22.4%,15.2%,46.9%,1.7%,0.2%,0.8%,11.4%


In [127]:
trip_all.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_county,,,,,,,,
King,"146,322","1,784,864","1,163,516","3,834,922","131,117","36,067","289,987","1,452,878"
Kitsap,"12,501","224,396","144,239","457,501","14,704","2,246","12,674","142,637"
Pierce,"45,291","759,075","511,559","1,484,376","62,389","7,818","24,914","397,596"
Snohomish,"40,153","707,529","480,330","1,479,694","52,767","6,255","26,409","359,863"


In [128]:
df_non_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_county,,,,,,,,
King,1.6%,22.0%,14.4%,39.3%,1.7%,0.3%,3.3%,17.3%
Kitsap,1.3%,23.9%,15.4%,41.7%,1.6%,0.2%,0.9%,15.0%
Pierce,1.4%,24.8%,16.8%,40.9%,2.1%,0.2%,0.7%,13.0%
Snohomish,1.3%,24.3%,16.5%,42.5%,1.9%,0.2%,0.9%,12.4%


In [129]:
trip_non_work.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_county,,,,,,,,
King,"125,096","1,671,559","1,094,146","2,981,285","131,102","26,303","253,283","1,310,037"
Kitsap,"11,390","213,530","137,417","373,330","14,697","2,017","7,976","134,566"
Pierce,"41,218","720,808","487,605","1,189,969","62,382","6,812","20,142","376,978"
Snohomish,"36,292","667,228","454,798","1,168,107","52,763","5,240","23,761","339,987"


In [130]:
df_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_county,,,,,,,,
King,1.7%,9.1%,5.6%,68.5%,0.0%,0.8%,2.9%,11.5%
Kitsap,1.0%,9.4%,5.9%,72.6%,0.0%,0.2%,4.1%,7.0%
Pierce,1.1%,9.9%,6.2%,76.1%,0.0%,0.3%,1.2%,5.3%
Snohomish,1.0%,10.0%,6.3%,77.0%,0.0%,0.3%,0.7%,4.9%


In [131]:
trip_work.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_county,,,,,,,,
King,"21,226","113,305","69,370","853,637",15,"9,764","36,704","142,841"
Kitsap,"1,111","10,866","6,822","84,171",7,229,"4,698","8,071"
Pierce,"4,073","38,267","23,954","294,407",7,"1,006","4,772","20,618"
Snohomish,"3,861","40,301","25,532","311,587",4,"1,015","2,648","19,876"


In [132]:
df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work  = calc_mode_share(trip_rgc, 'hh_rgc')

df_all

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rgc,,,,,,,,
All RGCs,3.0%,10.7%,5.2%,29.2%,0.5%,1.5%,6.5%,43.3%
Auburn,2.3%,19.4%,10.0%,38.4%,1.1%,0.4%,3.7%,24.7%
Bellevue,2.6%,11.4%,4.9%,31.8%,0.5%,0.8%,3.4%,44.5%
Bothell Canyon Park,1.2%,19.4%,10.6%,52.5%,1.4%,0.3%,0.8%,13.6%
Bremerton,2.5%,14.7%,7.2%,35.5%,0.4%,0.6%,2.3%,36.9%
Burien,2.0%,17.9%,9.2%,43.8%,0.7%,0.5%,3.1%,22.9%
Everett,1.9%,13.2%,6.9%,39.1%,0.7%,0.5%,2.1%,35.7%
Federal Way,2.5%,21.2%,15.0%,35.8%,1.8%,0.1%,1.5%,22.1%
Greater Downtown Kirkland,1.9%,17.3%,8.3%,47.1%,0.8%,0.5%,3.7%,20.4%


In [133]:
trip_all.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rgc,,,,,,,,
All RGCs,"33,808","119,795","57,965","325,585","5,837","16,822","73,060","483,754"
Auburn,200,"1,667",859,"3,294",94,37,314,"2,122"
Bellevue,"1,768","7,631","3,295","21,318",308,564,"2,298","29,871"
Bothell Canyon Park,32,509,278,"1,379",38,9,22,358
Bremerton,316,"1,887",932,"4,569",46,77,302,"4,747"
Burien,294,"2,697","1,392","6,585",111,71,461,"3,440"
Everett,495,"3,490","1,811","10,300",181,125,541,"9,412"
Federal Way,59,507,360,857,44,2,37,528
Greater Downtown Kirkland,605,"5,527","2,644","15,033",255,152,"1,176","6,522"


In [134]:
df_non_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rgc,,,,,,,,
All RGCs,3.0%,11.8%,5.8%,26.2%,0.6%,1.3%,6.9%,44.4%
Auburn,2.3%,20.6%,10.6%,35.3%,1.2%,0.4%,3.2%,26.4%
Bellevue,2.5%,12.5%,5.4%,28.3%,0.5%,0.7%,3.8%,46.2%
Bothell Canyon Park,1.2%,21.2%,11.6%,47.8%,1.7%,0.4%,0.8%,15.4%
Bremerton,2.4%,15.8%,7.7%,32.0%,0.4%,0.6%,1.6%,39.5%
Burien,1.9%,19.1%,9.9%,38.9%,0.9%,0.4%,3.2%,25.6%
Everett,1.8%,14.2%,7.5%,34.7%,0.8%,0.5%,2.2%,38.2%
Federal Way,2.5%,22.2%,16.0%,32.4%,2.1%,0.1%,1.7%,23.1%
Greater Downtown Kirkland,1.9%,18.8%,9.0%,42.8%,0.9%,0.5%,3.8%,22.4%


In [135]:
trip_non_work.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rgc,,,,,,,,
All RGCs,"27,604","109,549","53,317","243,144","5,837","11,612","64,365","411,180"
Auburn,172,"1,564",808,"2,683",94,32,242,"2,006"
Bellevue,"1,428","7,004","3,029","15,885",308,397,"2,115","25,937"
Bothell Canyon Park,27,471,257,"1,064",38,8,17,343
Bremerton,265,"1,777",866,"3,590",46,66,182,"4,428"
Burien,245,"2,459","1,273","4,999",111,53,414,"3,293"
Everett,421,"3,245","1,709","7,927",181,108,503,"8,725"
Federal Way,53,473,342,691,44,2,36,494
Greater Downtown Kirkland,509,"5,120","2,465","11,680",255,124,"1,024","6,102"


In [136]:
df_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rgc,,,,,,,,
All RGCs,3.3%,5.4%,2.4%,43.4%,NaN,2.7%,4.6%,38.2%
Auburn,2.8%,10.4%,5.2%,62.0%,NaN,0.5%,7.3%,11.8%
Bellevue,3.1%,5.7%,2.4%,49.6%,NaN,1.5%,1.7%,35.9%
Bothell Canyon Park,1.2%,9.5%,5.2%,78.8%,NaN,0.2%,1.2%,3.8%
Bremerton,3.1%,6.6%,4.0%,59.1%,NaN,0.7%,7.2%,19.3%
Burien,2.2%,10.8%,5.4%,72.0%,NaN,0.8%,2.1%,6.7%
Everett,2.1%,6.9%,2.9%,67.1%,NaN,0.5%,1.1%,19.4%
Federal Way,2.3%,13.1%,6.9%,64.1%,NaN,NaN,0.4%,13.1%
Greater Downtown Kirkland,2.1%,8.8%,3.9%,72.3%,NaN,0.6%,3.3%,9.1%


In [137]:
trip_work.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rgc,,,,,,,,
All RGCs,"6,204","10,246","4,648","82,441",nan,"5,210","8,695","72,574"
Auburn,28,103,51,611,nan,5,72,116
Bellevue,340,627,266,"5,433",nan,167,183,"3,934"
Bothell Canyon Park,5,38,21,315,nan,1,5,15
Bremerton,51,110,66,979,nan,11,120,319
Burien,49,238,119,"1,586",nan,18,47,147
Everett,74,245,102,"2,373",nan,17,38,687
Federal Way,6,34,18,166,nan,nan,1,34
Greater Downtown Kirkland,96,407,179,"3,353",nan,28,152,420


In [138]:
df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work  = calc_mode_share(trip_rg, 'hh_rg_proposed')

df_all

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rg_proposed,,,,,,,,
CitiesTowns,1.2%,22.7%,16.5%,45.1%,2.1%,0.2%,0.6%,11.7%
Core,1.4%,22.2%,14.8%,45.7%,1.7%,0.2%,1.8%,12.1%
HCT,1.3%,22.6%,15.3%,46.2%,1.8%,0.2%,1.4%,11.3%
Metro,2.1%,17.7%,10.7%,40.2%,1.2%,0.6%,4.4%,23.2%
UU,1.2%,24.2%,17.8%,44.8%,2.1%,0.2%,0.5%,9.3%


In [139]:
trip_all.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rg_proposed,,,,,,,,
CitiesTowns,"16,444","322,877","234,656","639,961","29,559","2,309","8,194","166,502"
Core,"53,219","834,916","557,406","1,717,940","64,206","8,938","68,470","455,049"
HCT,"46,257","803,786","543,550","1,643,060","63,816","7,736","48,333","400,683"
Metro,"103,486","882,296","530,717","1,999,602","58,261","30,204","218,425","1,154,820"
UU,"7,807","156,034","114,605","289,155","13,646","1,029","2,970","59,718"


In [140]:
df_non_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rg_proposed,,,,,,,,
CitiesTowns,1.2%,24.5%,17.9%,40.6%,2.4%,0.2%,0.5%,12.7%
Core,1.4%,24.0%,16.1%,41.4%,2.0%,0.2%,1.8%,13.1%
HCT,1.3%,24.4%,16.6%,41.9%,2.1%,0.2%,1.3%,12.2%
Metro,2.0%,19.4%,11.7%,36.2%,1.4%,0.5%,4.5%,24.4%
UU,1.3%,26.0%,19.2%,40.5%,2.4%,0.1%,0.4%,10.0%


In [141]:
trip_non_work.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rg_proposed,,,,,,,,
CitiesTowns,"15,316","304,575","222,475","505,354","29,552","1,915","6,683","157,894"
Core,"47,145","785,902","526,183","1,351,924","64,199","7,328","58,894","426,682"
HCT,"41,752","758,551","514,853","1,300,714","63,805","6,446","40,878","378,305"
Metro,"86,423","822,110","496,730","1,535,538","58,259","22,042","191,102","1,034,256"
UU,"7,313","148,293","109,535","230,668","13,646",851,"2,264","56,919"


In [142]:
df_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rg_proposed,,,,,,,,
CitiesTowns,0.6%,10.4%,6.9%,76.2%,0.0%,0.2%,0.9%,4.9%
Core,1.2%,10.0%,6.3%,74.4%,0.0%,0.3%,1.9%,5.8%
HCT,1.0%,10.0%,6.4%,75.8%,0.0%,0.3%,1.6%,5.0%
Metro,2.3%,8.2%,4.6%,63.5%,0.0%,1.1%,3.7%,16.5%
UU,0.7%,10.3%,6.7%,77.5%,NaN,0.2%,0.9%,3.7%


In [143]:
trip_work.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rg_proposed,,,,,,,,
CitiesTowns,"1,128","18,302","12,181","134,607",7,394,"1,511","8,608"
Core,"6,074","49,014","31,223","366,016",7,"1,610","9,576","28,367"
HCT,"4,505","45,235","28,697","342,346",11,"1,290","7,455","22,378"
Metro,"17,063","60,186","33,987","464,064",2,"8,162","27,323","120,564"
UU,494,"7,741","5,070","58,487",nan,178,706,"2,799"


In [144]:
equity_geogs = summary_config['equity_geogs']

trip_equity_geog = pd.DataFrame()

for geog in equity_geogs:
    df = prep_trip_df(pd.read_csv(output_path/ ('agg/dash/mode_share_'+geog+'.csv')))
    df['equity_geog'] = df[geog].map({1:geog,
                                      0:"NOT in "+ geog})
    
    trip_equity_geog = pd.concat([trip_equity_geog, df[['dpurp', 'Mode', 'equity_geog', 'trexpfac', 'Trip Type']]])

df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work  = calc_mode_share(trip_equity_geog, 'equity_geog')

In [145]:
df_all

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
equity_geog,,,,,,,,
NOT in hh_disability_reg,1.5%,21.4%,14.5%,44.9%,1.7%,0.3%,2.2%,13.6%
NOT in hh_elderly_reg,1.6%,20.8%,14.3%,43.5%,1.7%,0.3%,2.3%,15.4%
NOT in hh_english_reg,1.5%,21.1%,13.7%,44.7%,1.5%,0.3%,2.1%,15.0%
NOT in hh_poverty_50,1.5%,21.4%,14.1%,44.7%,1.6%,0.3%,2.1%,14.3%
NOT in hh_poverty_reg,1.4%,21.5%,14.3%,45.6%,1.6%,0.3%,1.9%,13.4%
NOT in hh_racial_50,1.4%,21.4%,14.0%,45.5%,1.6%,0.3%,1.9%,13.9%
NOT in hh_racial_reg,1.4%,21.5%,13.9%,46.0%,1.5%,0.3%,1.9%,13.5%
NOT in hh_youth_reg,1.7%,19.7%,12.0%,43.7%,1.2%,0.5%,3.1%,18.1%
hh_disability_reg,1.5%,21.3%,13.6%,44.1%,1.5%,0.4%,2.1%,15.5%


In [146]:
trip_all.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
equity_geog,,,,,,,,
NOT in hh_disability_reg,"135,275","1,946,360","1,321,883","4,090,757","153,979","26,377","202,048","1,236,700"
NOT in hh_elderly_reg,"143,093","1,814,028","1,243,848","3,783,933","148,628","29,738","203,704","1,337,764"
NOT in hh_english_reg,"149,218","2,102,978","1,369,276","4,455,696","153,930","33,651","209,089","1,497,973"
NOT in hh_poverty_50,"236,952","3,419,212","2,261,444","7,151,967","256,847","50,770","335,695","2,285,070"
NOT in hh_poverty_reg,"138,735","2,129,263","1,415,374","4,518,339","160,256","28,930","189,423","1,324,157"
NOT in hh_racial_50,"158,004","2,377,653","1,552,793","5,045,771","173,756","32,101","212,737","1,540,500"
NOT in hh_racial_reg,"120,427","1,871,853","1,214,339","4,008,614","134,232","23,368","164,079","1,176,896"
NOT in hh_youth_reg,"142,894","1,635,246","997,246","3,626,500","97,855","37,395","253,779","1,500,872"
hh_disability_reg,"108,992","1,529,508","977,769","3,165,746","106,998","26,009","151,936","1,116,274"


In [147]:
df_non_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
equity_geog,,,,,,,,
NOT in hh_disability_reg,1.5%,23.2%,15.9%,40.6%,2.0%,0.2%,2.2%,14.4%
NOT in hh_elderly_reg,1.7%,22.7%,15.7%,39.1%,2.0%,0.3%,2.3%,16.3%
NOT in hh_english_reg,1.5%,22.9%,15.0%,40.7%,1.8%,0.3%,2.0%,15.9%
NOT in hh_poverty_50,1.5%,23.2%,15.4%,40.6%,1.9%,0.3%,2.1%,15.1%
NOT in hh_poverty_reg,1.4%,23.4%,15.6%,41.4%,1.9%,0.3%,1.9%,14.1%
NOT in hh_racial_50,1.4%,23.2%,15.2%,41.4%,1.8%,0.3%,1.9%,14.8%
NOT in hh_racial_reg,1.4%,23.3%,15.2%,41.9%,1.8%,0.2%,1.8%,14.4%
NOT in hh_youth_reg,1.7%,21.5%,13.1%,39.8%,1.4%,0.4%,3.1%,19.0%
hh_disability_reg,1.5%,23.0%,14.7%,40.1%,1.7%,0.3%,2.1%,16.4%


In [148]:
trip_non_work.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
equity_geog,,,,,,,,
NOT in hh_disability_reg,"118,008","1,829,031","1,249,007","3,193,867","153,957","19,628","172,723","1,131,127"
NOT in hh_elderly_reg,"124,235","1,701,428","1,174,213","2,934,096","148,609","22,218","175,550","1,221,833"
NOT in hh_english_reg,"130,569","1,982,603","1,295,647","3,520,474","153,908","25,759","176,906","1,374,233"
NOT in hh_poverty_50,"207,567","3,219,434","2,137,487","5,630,107","256,814","39,080","288,707","2,098,997"
NOT in hh_poverty_reg,"121,576","2,004,163","1,337,835","3,553,345","160,231","21,757","161,396","1,212,615"
NOT in hh_racial_50,"139,071","2,241,230","1,469,069","3,989,048","173,731","25,191","179,971","1,425,221"
NOT in hh_racial_reg,"106,146","1,764,872","1,148,623","3,173,991","134,211","18,513","137,463","1,093,362"
NOT in hh_youth_reg,"121,807","1,533,619","937,761","2,838,497","97,843","28,162","219,149","1,358,932"
hh_disability_reg,"95,988","1,444,098","924,967","2,518,833","106,987","20,744","132,439","1,030,441"


In [149]:
df_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
equity_geog,,,,,,,,
NOT in hh_disability_reg,1.4%,9.4%,5.8%,72.0%,0.0%,0.5%,2.4%,8.5%
NOT in hh_elderly_reg,1.6%,9.4%,5.8%,70.7%,0.0%,0.6%,2.3%,9.6%
NOT in hh_english_reg,1.4%,9.2%,5.6%,71.3%,0.0%,0.6%,2.5%,9.4%
NOT in hh_poverty_50,1.4%,9.4%,5.8%,71.8%,0.0%,0.6%,2.2%,8.8%
NOT in hh_poverty_reg,1.3%,9.4%,5.8%,72.5%,0.0%,0.5%,2.1%,8.4%
NOT in hh_racial_50,1.3%,9.4%,5.8%,72.8%,0.0%,0.5%,2.3%,7.9%
NOT in hh_racial_reg,1.3%,9.4%,5.8%,73.4%,0.0%,0.4%,2.3%,7.3%
NOT in hh_youth_reg,1.8%,8.8%,5.1%,68.2%,0.0%,0.8%,3.0%,12.3%
hh_disability_reg,1.4%,9.4%,5.8%,71.2%,0.0%,0.6%,2.1%,9.4%


In [150]:
trip_work.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
equity_geog,,,,,,,,
NOT in hh_disability_reg,"17,267","117,329","72,876","896,890",22,"6,749","29,325","105,573"
NOT in hh_elderly_reg,"18,858","112,600","69,635","849,837",19,"7,520","28,154","115,931"
NOT in hh_english_reg,"18,649","120,375","73,629","935,222",22,"7,892","32,183","123,740"
NOT in hh_poverty_50,"29,385","199,778","123,957","1,521,860",33,"11,690","46,988","186,073"
NOT in hh_poverty_reg,"17,159","125,100","77,539","964,994",25,"7,173","28,027","111,542"
NOT in hh_racial_50,"18,933","136,423","83,724","1,056,723",25,"6,910","32,766","115,279"
NOT in hh_racial_reg,"14,281","106,981","65,716","834,623",21,"4,855","26,616","83,534"
NOT in hh_youth_reg,"21,087","101,627","59,485","788,003",12,"9,233","34,630","141,940"
hh_disability_reg,"13,004","85,410","52,802","646,913",11,"5,265","19,497","85,833"


## mode share by trip destination

In [151]:
tour_rgc_dest = pd.read_csv(output_path / 'agg/dash/tour_rgc_dest.csv')
trip_rgc_dest = pd.read_csv(output_path / 'agg/dash/trip_rgc_dest.csv')

# order RGCs

# all individual centers
l = tour_rgc_dest['tour_d_rgc'].sort_values().unique().tolist()
l.remove('Not in RGC')
l.remove(np.nan)

# aggregated centers
geog_order = ["Region","In RGC","Not in RGC"]

# combine
geog_order.extend(l)

In [152]:
def get_rgc_location(df, dest_col):
    # regional data
    df_region = df.copy()
    df_region[dest_col] = "Region"

    # aggregated centers
    df_isrgc = df.copy()
    df_isrgc.loc[df_isrgc[dest_col]!="Not in RGC", dest_col] = "In RGC"


    df_rgc = pd.concat([
        df_region,
        df_isrgc,
    # 2025 notes:
    # 7 records in the data are missing both mode and and RGC values
    # remove these records
        df.loc[~df[dest_col].isna()]
        ])

    df_rgc[dest_col] = pd.Categorical(df_rgc[dest_col], ordered=True,
                    categories=geog_order)
    
    return df_rgc

df_tour = get_rgc_location(tour_rgc_dest, 'tour_d_rgc')
df_trip = get_rgc_location(trip_rgc_dest, 'trip_d_rgc')




In [153]:
def calc_mode_share_by_rgc(df_trip_geog, geog_col, mode_col, n_trip_col):

    df = df_trip_geog.pivot_table(columns=mode_col, index=geog_col, values=n_trip_col, aggfunc='sum', observed=True)

    df = df.apply(lambda x: x/ df_trip_geog.groupby(geog_col, observed=True)[n_trip_col].sum())

    return df

\[TRIP\] Mode Share for All Purposes by Regional Center (Destination Location)

In [154]:
calc_mode_share_by_rgc(df_trip, 'trip_d_rgc', 'mode', 'trexpfac')

mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
trip_d_rgc,,,,,,,,
Region,1.5%,21.3%,14.1%,44.5%,1.6%,0.3%,2.2%,14.4%
In RGC,1.9%,15.9%,8.9%,42.0%,0.5%,0.8%,5.6%,24.5%
Not in RGC,1.4%,22.4%,15.1%,45.0%,1.8%,0.2%,1.5%,12.5%
Auburn,1.1%,22.3%,12.5%,48.2%,0.5%,0.2%,3.2%,12.0%
Bellevue,1.7%,15.0%,8.8%,42.9%,0.4%,0.5%,2.5%,28.2%
Bothell Canyon Park,1.0%,22.3%,14.2%,54.0%,1.0%,0.2%,0.4%,6.8%
Bremerton,1.6%,15.7%,9.0%,54.4%,0.1%,0.3%,1.6%,17.3%
Burien,1.2%,22.6%,12.5%,46.4%,0.8%,0.2%,2.2%,14.1%
Everett,1.2%,19.1%,11.0%,47.3%,0.6%,0.3%,1.6%,18.9%


\[TRIP\] Mode Share to Work by Regional Center (Destination Location)

In [155]:
df_trip_work = df_trip[df_trip['dpurp'] == 'Work'].copy()
calc_mode_share_by_rgc(df_trip_work, 'trip_d_rgc', 'mode', 'trexpfac')

mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
trip_d_rgc,,,,,,,,
Region,1.4%,9.4%,5.8%,71.6%,0.0%,0.6%,2.3%,8.9%
In RGC,1.8%,8.6%,5.4%,61.9%,0.0%,1.0%,4.9%,16.5%
Not in RGC,1.2%,9.8%,6.1%,76.7%,0.0%,0.3%,0.9%,5.0%
Auburn,1.1%,10.3%,6.9%,73.5%,NaN,0.2%,1.1%,7.0%
Bellevue,1.5%,9.1%,6.0%,64.9%,NaN,0.6%,1.8%,16.1%
Bothell Canyon Park,1.0%,10.0%,5.7%,79.3%,NaN,0.3%,0.2%,3.5%
Bremerton,1.1%,9.9%,6.3%,75.0%,NaN,0.3%,1.3%,6.1%
Burien,1.5%,9.8%,6.4%,74.2%,NaN,0.4%,1.0%,6.7%
Everett,1.3%,8.9%,5.6%,72.4%,0.0%,0.3%,0.6%,10.9%


\[TRIP\] Mode Share for Non-work Purposes by Regional Center (Destination Location)

In [156]:
df_trip_non_work = df_trip[df_trip['dpurp'] != 'Work'].copy()
calc_mode_share_by_rgc(df_trip_non_work, 'trip_d_rgc', 'mode', 'trexpfac')

mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
trip_d_rgc,,,,,,,,
Region,1.5%,23.1%,15.4%,40.4%,1.8%,0.3%,2.2%,15.3%
In RGC,1.9%,18.8%,10.3%,34.4%,0.6%,0.7%,5.8%,27.5%
Not in RGC,1.5%,23.8%,16.2%,41.3%,2.0%,0.2%,1.6%,13.4%
Auburn,1.1%,24.6%,13.5%,43.4%,0.6%,0.2%,3.6%,13.0%
Bellevue,1.8%,17.9%,10.1%,32.2%,0.6%,0.4%,2.9%,34.1%
Bothell Canyon Park,1.0%,27.3%,17.7%,43.7%,1.5%,0.2%,0.5%,8.2%
Bremerton,1.9%,21.1%,11.5%,35.4%,0.2%,0.3%,1.9%,27.6%
Burien,1.1%,24.0%,13.2%,43.3%,0.9%,0.2%,2.3%,14.9%
Everett,1.2%,21.8%,12.5%,40.6%,0.7%,0.3%,1.9%,21.0%


## mode share by tour destination

\[TOUR\] Mode Share for All Purposes by Regional Center (Destination Location)

In [157]:
calc_mode_share_by_rgc(df_tour, 'tour_d_rgc', 'tmodetp', 'toexpfac')

tmodetp,Bike,HOV2,HOV3+,Park,SOV,School Bus,TNC,Transit,Walk
tour_d_rgc,,,,,,,,,
Region,1.4%,22.1%,19.3%,0.2%,37.9%,2.6%,0.5%,3.2%,13.0%
In RGC,1.5%,18.4%,14.2%,0.5%,39.4%,0.6%,1.0%,7.9%,16.3%
Not in RGC,1.3%,23.1%,20.7%,0.1%,37.5%,3.1%,0.3%,1.9%,12.0%
Auburn,0.7%,25.1%,20.9%,0.1%,41.9%,0.3%,0.3%,2.3%,8.4%
Bellevue,1.3%,18.0%,14.4%,0.3%,42.6%,0.3%,0.7%,3.4%,19.1%
Bothell Canyon Park,0.9%,24.0%,21.6%,0.0%,45.3%,1.9%,0.3%,0.9%,5.1%
Bremerton,1.2%,21.0%,15.8%,0.3%,50.0%,0.1%,0.4%,0.8%,10.3%
Burien,0.8%,23.6%,18.2%,0.0%,40.2%,1.6%,0.3%,3.8%,11.6%
Everett,0.9%,22.8%,18.0%,0.1%,40.3%,1.0%,0.4%,2.5%,14.0%


\[TOUR\] Mode Share to Work by Regional Center (Destination Location)

In [158]:
df_tour_work = df_tour[df_tour['pdpurp'] == 'Work'].copy()
calc_mode_share_by_rgc(df_tour_work, 'tour_d_rgc', 'tmodetp', 'toexpfac')

tmodetp,Bike,HOV2,HOV3+,Park,SOV,TNC,Transit,Walk
tour_d_rgc,,,,,,,,
Region,1.4%,15.3%,12.4%,0.5%,62.3%,1.0%,2.7%,4.3%
In RGC,1.8%,13.9%,11.3%,1.1%,55.2%,1.8%,6.0%,8.8%
Not in RGC,1.2%,16.1%,13.1%,0.2%,66.1%,0.6%,1.0%,1.8%
Auburn,1.2%,16.8%,15.1%,0.3%,62.5%,0.4%,1.3%,2.3%
Bellevue,1.5%,15.5%,13.1%,0.5%,58.9%,1.0%,2.0%,7.5%
Bothell Canyon Park,0.9%,15.8%,12.5%,0.1%,69.2%,0.6%,0.3%,0.6%
Bremerton,0.9%,17.0%,13.7%,0.5%,64.6%,0.5%,1.2%,1.7%
Burien,1.3%,15.7%,13.1%,0.1%,65.1%,0.5%,1.3%,2.8%
Everett,1.4%,16.3%,13.2%,0.2%,62.4%,0.7%,0.7%,5.2%


\[TOUR\] Mode Share for Non-work Purposes by Regional Center (Destination Location)

In [159]:
df_tour_non_work = df_tour[df_tour['pdpurp'] != 'Work'].copy()
calc_mode_share_by_rgc(df_tour_non_work, 'tour_d_rgc', 'tmodetp', 'toexpfac')

tmodetp,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
tour_d_rgc,,,,,,,,
Region,1.3%,24.7%,21.9%,28.6%,3.5%,0.2%,3.4%,16.3%
In RGC,1.3%,21.9%,16.5%,27.2%,1.1%,0.4%,9.5%,22.1%
Not in RGC,1.3%,25.3%,23.0%,28.9%,4.1%,0.2%,2.2%,15.0%
Auburn,0.6%,28.1%,23.0%,34.5%,0.4%,0.2%,2.7%,10.6%
Bellevue,1.0%,21.0%,16.1%,22.0%,0.7%,0.3%,5.2%,33.8%
Bothell Canyon Park,0.8%,29.4%,27.5%,29.8%,3.1%,0.2%,1.2%,8.0%
Bremerton,1.7%,27.1%,19.1%,27.5%,0.3%,0.3%,0.2%,23.7%
Burien,0.7%,25.2%,19.2%,35.2%,1.9%,0.2%,4.3%,13.3%
Everett,0.8%,25.6%,20.0%,31.1%,1.4%,0.3%,3.3%,17.7%
